In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Trendlijn voor gebruik gewoon over de jaren

In [14]:
df_origineel = pd.read_excel('Totaal_gebruik.xlsx')
df_origineel.head()

,Year,Snorfiets,Bromfiets,Totaal
0,2019,716917,387331,1104248
1,2020,728644,388345,1116989
2,2021,763342,393991,1157333
3,2022,781353,404282,1185635
4,2023,761219,436996,1198215


In [18]:
df = df_origineel.rename(columns={
    "Year": "Year",
    "Snorfiets": "Snorfiets",
    "Bromfiets": "Bromfiets"
})[["Year", "Snorfiets", "Bromfiets"]]

long = df.melt(id_vars="Year",
                value_vars=["Snorfiets", "Bromfiets"],
                var_name="Serie", value_name="Aantal")


fig = px.scatter(
    long, x="Year", y="Aantal", color="Serie", trendline = "ols",
    labels={"Year": "Jaar", "Aantal": "Aantal", "Serie": ""},
)
fig.update_traces(hovertemplate="Jaar %{x}<br> Aantal %{y:.0f}")
fig.update_layout(title="Aantal gebruikers — Bromfiets vs. Snorfiets", xaxis=dict(dtick=1))
fig.show()


## info over de trendlijn

In [5]:
# haal regressieresultaten uit fig
res = px.get_trendline_results(fig)

# bepaal kolomnaam waarin de modellen zitten
col_model = "px_fit_results" if "px_fit_results" in res.columns else "results"

# bepaal kolomnaam voor de groepsnaam
col_group = "Serie" if "Serie" in res.columns else ("color" if "color" in res.columns else res.columns[0])

# veilige extractie van de parameters
rows = []
for i, row in res.iterrows():
    model = row[col_model]
    try:
        params = model.params
        # Zorg dat we altijd labels hebben
        if isinstance(params, (list, tuple, np.ndarray)):
            a, b = params
        else:
            a = params.get("const", list(params)[0] if len(params) > 0 else np.nan)
            b = params.get("Year", list(params)[-1] if len(params) > 1 else np.nan)
        rows.append({
            "Serie": row.get(col_group, f"groep_{i}"),
            "intercept": float(a),
            "slope_per_jaar": float(b),
            "R2": float(model.rsquared)
        })
    except Exception as e:
        print(f"⚠️ Skipping group {i}: {e}")

summary = pd.DataFrame(rows)
print(summary)

       Serie     intercept  slope_per_jaar        R2
0  Snorfiets  3.671116e+07   -17801.857143  0.337696
1  Bromfiets -4.748444e+07    23697.928571  0.854380


R2 is dus hoe goed de voorspelling is. en dan intercept is de constante (a)

y = a + bx 

slope per jaar is dus de b 

x is dus jaar 

dus op basis van deze gegevens zou je voorspellingen voor volgende jaren kunnen maken.

# Trendlijn gesplits met voor en na invoer helmplicht

In [7]:
# 1) Jouw df -> long
long = df.melt(id_vars="Year",
               value_vars=["Snorfiets", "Bromfiets"],
               var_name="Serie", value_name="Aantal")

# 2) Zorg dat Year/Aantal numeriek zijn
long["Year"] = pd.to_numeric(long["Year"], errors="coerce")
long["Aantal"] = pd.to_numeric(long["Aantal"], errors="coerce")

# 3) Periode-labels + gecombineerde groep
long["Periode"] = np.where(long["Year"] <= 2022, "Voor helmplicht", "Na helmplicht")
long["Groep"] = long["Serie"] + " — " + long["Periode"]

# 4) Plot met OLS-trendlijnen per groep
fig = px.scatter(
    long,
    x="Year", y="Aantal",
    color="Groep",               # 4 groepen => 4 trendlijnen
    trendline="ols",
    trendline_scope="trace",     # per kleur-trace
    labels={"Year": "Jaar", "Aantal": "Aantal gebruikers"},
    title="Aantal gebruikers — Bromfiets vs. Snorfiets (voor en na helmplicht)"
)
fig.add_vline(x=2023, line_dash="dash", line_color="red",
              annotation_text="Helmet law inplemented ", annotation_position="top right")


fig.update_layout(xaxis=dict(dtick=1))
fig.show()

## Info over de trendlijn

In [8]:
# haal regressieresultaten uit fig
res = px.get_trendline_results(fig)

# bepaal kolomnaam waarin de modellen zitten
col_model = "px_fit_results" if "px_fit_results" in res.columns else "results"

# bepaal kolomnaam voor de groepsnaam
col_group = "Serie" if "Serie" in res.columns else ("color" if "color" in res.columns else res.columns[0])

# veilige extractie van de parameters
rows = []
for i, row in res.iterrows():
    model = row[col_model]
    try:
        params = model.params
        # Zorg dat we altijd labels hebben
        if isinstance(params, (list, tuple, np.ndarray)):
            a, b = params
        else:
            a = params.get("const", list(params)[0] if len(params) > 0 else np.nan)
            b = params.get("Year", list(params)[-1] if len(params) > 1 else np.nan)
        rows.append({
            "Serie": row.get(col_group, f"groep_{i}"),
            "intercept": float(a),
            "slope_per_jaar": float(b),
            "R2": float(model.rsquared)
        })
    except Exception as e:
        print(f"⚠️ Skipping group {i}: {e}")

summary = pd.DataFrame(rows)
print(summary)

                         Serie     intercept  slope_per_jaar        R2
0  Snorfiets — Voor helmplicht -4.532105e+07         22800.6  0.967070
1    Snorfiets — Na helmplicht  1.705427e+08        -83927.5  0.993799
2  Bromfiets — Voor helmplicht -1.102214e+07          5649.9  0.881209
3    Bromfiets — Na helmplicht -8.591537e+07         42688.0  0.953282


# Voorspelling voor jaren 2026 - 2030

In [19]:
years = [ 2026, 2027, 2028, 2029, 2030]

#y = a + bx 
a_snor = int(summary.loc[1, 'intercept'])
a_brom = int(summary.loc[3, 'intercept'])
b_snor = int(summary.loc[1, 'slope_per_jaar'])
b_brom = int(summary.loc[3, 'slope_per_jaar'])

y_snor = []
y_brom = []


for i in years:
    y_snor.append(a_snor + b_snor*i)
    y_brom.append(a_brom + b_brom*i)

df_future = pd.DataFrame({
    "Year": years,
    "Snorfiets": y_snor,
    "Bromfiets": y_brom
})

df_future["Totaal"] = df_future["Snorfiets"] + df_future["Bromfiets"]

df_combined = pd.concat([df_origineel, df_future], ignore_index=True)


In [20]:
fig = px.line(df_combined, x="Year", y=["Snorfiets", "Bromfiets"], markers=True,
              labels={"value": "Aantal", "variable": "Type", "Year": "Jaar"},
              title="Aantal voorspeld per jaar")
fig.show()
